16.06.2022

# Correzione manuale degli errori nelle masks

- comparare labels RGB e labels con class values {0,1,2,3,4}
- trovare i cambiamenti manuali (che non sono presenti nei labels RGB)
- aprire i labels con Napari e associare tutti i cambiamenti manuali all'evento corretto
- sempre in Napari, aggiungere altre correzioni manuali 
- dalla nuova array, creare file .csv tale che ogni evento è come segue:
    - "event; x_array; y_array; Frame"
    - e.g. "32; 1,3,7,8,10; 3,4,3,5,7; 20" (<- counting from 0) (len(x_array) == len(y_array))
- salvare come .csv
- mandare a Miguel

In [34]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import os
import imageio

from scipy.signal import detrend
import numpy as np
import cc3d

import napari
from matplotlib import cm
import vispy.color

from data.data_processing_tools import moving_average, count_classes_in_roi
from utils.in_out_tools import (
    load_movies_ids,
    load_annotations_ids,
    load_rgb_annotations_ids,
)
from utils.visualization_tools import get_discrete_cmap

In [36]:
movie_ids = ["34"]
# movie_ids = ["01","02","03","04","05","06","07","08","09",
#             "10","11","12","13","14","15","16","17","18","19",
#             "20","21","22","23","24","25","27","28","29",
#             "30","32","33","34","35","36","38","39",
#             "40","41","42","43","44","45","46"
#            ]

In [37]:
data_dir = os.path.join("..", "data", "raw_data_and_processing")

### directory where corrected masks are saved

In [38]:
# directory where corrected movies are saved
corr_dir = os.path.join(data_dir, "manual_corr_separated_event_masks")
os.makedirs(corr_dir, exist_ok=True)

### load original movies

In [39]:
movies_dir = os.path.join(data_dir, "original_movies")
movies = load_movies_ids(data_folder=movies_dir, ids=movie_ids)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


### load smooth movies

In [40]:
smooth_movies_dir = os.path.join(data_dir, "smoothed_movies")
smooth_movies = load_movies_ids(
    data_folder=smooth_movies_dir,
    ids=movie_ids,
    names_available=True,
    movie_names="smoothed_video",
)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


### load class annotations

In [41]:
class_labels_dir = os.path.join(data_dir, "original_masks")
class_labels = load_annotations_ids(
    data_folder=class_labels_dir, ids=movie_ids, mask_names="mask"
)

### General tools 

#### Create LUT for smooth movie

In [43]:
cmap = get_discrete_cmap(name="gray", lut=16)

C:\Users\dotti\AppData\Local\Temp\ipykernel_21520\249267619.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  segmented_cmap = cm.get_cmap(name=name, lut=16)


## Load original RBG masks and correct them

Compute diff = "bool"(class_labels)- "bool"(rgb_labels), if:
- diff == 0 then class_labels = rgb_labels
- diff == 1 then some ROIs have been added to the RGB labels in the class labels ==> set rgb_labels to white_int (i.e., add ROI)
- diff == -1 then some ROIs of the rgb labels have been removed in the class labels ==> set rgb_labels to 0 (i.e., remove ROI)

Afterwards, ROIs in rgb_labels with value === white_int have been adapted in the following way:
- if it is a new event, set ROI to a new colour
- if it belongs to an existing event, set ROI colour to the colour of the events to which it belongs

New masks are then saved (as "int"-RGB masks) in the directory `data/raw_data_and_processing_manual_corr_separated_events_masks` with name `XX_corrected_rgb_mask_V0.tif`

### load RGB annotations

In [31]:
rgb_labels_dir = os.path.join(data_dir, "original_separated_events_masks")
rgb_labels = load_rgb_annotations_ids(
    data_folder=rgb_labels_dir, ids=movie_ids, mask_names="separated_events"
)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


### compare rgb and class annotations and correct them

In [32]:
white_int = 255 * 255 * 255 + 255 * 255 + 255
print("white colour:", white_int)

white colour: 16646655


In [33]:
# check how many corrections are necessary in general

for movie_id in movie_ids:
    print("Movie", movie_id)
    rgb_label = rgb_labels[movie_id]
    class_label = class_labels[movie_id]

    # compute differences between rgb and class labels
    diff = np.where(class_label, 1, 0) - np.where(rgb_label, 1, 0)

    print("Frames containing differences:", *np.unique(np.where(diff)[0]))
    print("Values in the differences:", *np.unique(diff))

    # remove ROIs from rgb labels that are not present in class labels
    rgb_label_new = np.where(diff != -1, rgb_label, 0)

    # check differences after removing ROIs
    diff_2 = np.where(class_label, 1, 0) - np.where(rgb_label_new, 1, 0)
    print(
        "Frames containing differences after removing ROIs:",
        *np.unique(np.where(diff_2)[0])
    )
    # if diff_2.max() > 0:
    #    print("Movie", movie_id)
    #    print("Frames that need manual change of colour of ROI in rgb label:", *np.unique(np.where(diff_2)[0]))

    # add ROIs to rgb labels that are only present in class labels
    rgb_label_new = np.where(diff == 1, white_int, rgb_label_new)

    # check differences after adding ROIs
    diff_3 = np.where(class_label, 1, 0) - np.where(rgb_label_new, 1, 0)
    print(
        "Frames containing differences after adding ROIs:",
        *np.unique(np.where(diff_3)[0])
    )

    # update rgb_labels
    rgb_labels[movie_id] = rgb_label_new

Movie 34
Frames containing differences: 3 4 5 6 7 8 9 10 11 12 13 14 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 237 238 239 240 241 242 243 250 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 392 393 395 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542

### Correct single movies manually

In [15]:
movie_id = "34"
movie = movies[movie_id]
rgb_label = rgb_labels[movie_id]
class_label = class_labels[movie_id]

In [16]:
print(
    "Frames that need manual change of colour of ROI in rgb label:",
    *np.unique(np.where(rgb_label == white_int)[0])
)

Frames that need manual change of colour of ROI in rgb label: 352 364 366 382 485 523 542 711 712


In [17]:
rgb_label_colours = list(np.unique(rgb_label))
rgb_label_colours.remove(0)
rgb_label_colours.remove(white_int)

print("Colours present in rgb labels:", *rgb_label_colours)

Colours present in rgb labels: 286395 503758 1145893 1834601 2283011 3297477 4298938 4544150 6216714 8438263 10282145 10538020 10573481 11054862 12271355 12282243 13520126 13710565 14335114 14608866 14848626 15492916 15528586 15826204 16145127 16380485


### !!! the changes here are made interactively on the Napari viewer !!!

In [20]:
# os.environ['QT_PLUGIN_PATH'] = "C:\Users\dotti\.conda\envs\sparks\Library\plugins"

In [19]:
viewer = napari.Viewer()
viewer.add_image(movie, name="original_movie")
viewer.add_labels(rgb_label, name="rgb labels", opacity=0.5)
viewer.add_labels(class_label, name="class labels", opacity=0.5)
# viewer.add_labels(diff, name='difference between labels', opacity=0.5)

<Labels layer 'class labels' at 0x25b09640f10>

In [402]:
print(
    "Frames that need manual change of colour of ROI in rgb label after manual change of colours:",
    *np.unique(np.where(rgb_label == white_int)[0])
)

Frames that need manual change of colour of ROI in rgb label after manual change of colours:


In [416]:
# save corrected rgb labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V0.tif"), rgb_label
)

## Load corrected "int"-RGB labels and include additional manual corrections

For all movies, load file `data/raw_data_and_processing_manual_corr_separated_events_masks/XX_corrected_rgb_mask_V0.tif` and get colour of events that consist of more than one connected component (CC).

For each event that contain more than one CC, check them manually using _Napari_.

All the changes are tracked in the excel file `data/raw_data_and_processing_manual_corr_separated_events_masks/separated_events_analysis.xlsx`

Possible changes are, e.g.:
- delete some CCs
- change event/colour of some CCs
- join CCs

New masks are saved (as "int"-RGB masks) in the directory `data/raw_data_and_processing_manual_corr_separated_events_masks` with name `XX_corrected_rgb_mask_V1.tif`

In [10]:
rgb_labels_V0 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V0"
)

In [11]:
white_int = 255 * 255 * 255 + 255 * 255 + 255
print("white colour:", white_int)

white colour: 16646655


### check if int rgb label events consist of more than one connected component

In [12]:
# connectivity of the connected components
connectivity = 26

In [13]:
# check how many corrections are necessary in general

separated_events = {}

for movie_id in movie_ids:
    print("Movie", movie_id)
    int_rgb_label = rgb_labels_V0[movie_id]
    class_label = class_labels[movie_id]

    # check that int rgs label does not contain white_int
    assert not white_int in int_rgb_label

    # compute number of connected components for each event
    list_events = list(np.unique(int_rgb_label))
    list_events.remove(0)

    # create dict entry to store events that are separated in more than one conn comp
    separated_events[movie_id] = []

    for nb_event in list_events:
        events, n_components = cc3d.connected_components(
            (int_rgb_label == nb_event), connectivity=connectivity, return_N=True
        )
        if n_components > 1:
            print(
                f"Event with colour {nb_event} contains {n_components} connected components"
            )
            separated_events[movie_id].append([nb_event, n_components])

Movie 01
Event with colour 1460377 contains 2 connected components
Event with colour 1763421 contains 2 connected components
Event with colour 7235208 contains 2 connected components
Movie 02
Event with colour 399774 contains 2 connected components
Event with colour 583189 contains 3 connected components
Event with colour 4192011 contains 2 connected components
Event with colour 10252693 contains 7 connected components
Movie 03
Event with colour 83436 contains 4 connected components
Event with colour 212722 contains 2 connected components
Event with colour 2132871 contains 2 connected components
Event with colour 6204449 contains 2 connected components
Event with colour 8355437 contains 2 connected components
Event with colour 8587972 contains 3 connected components
Event with colour 11404912 contains 10 connected components
Event with colour 15234816 contains 3 connected components
Event with colour 15814187 contains 5 connected components
Event with colour 16063153 contains 3 connect

### Correct single movies manually

In [39]:
movie_id = "32"
movie = movies[movie_id]
int_rgb_label = rgb_labels_V0[movie_id]
class_label = class_labels[movie_id]

In [40]:
int_rgb_label_colours = list(np.unique(int_rgb_label))
int_rgb_label_colours.remove(0)

print("Colours present in rgb labels:", *int_rgb_label_colours)

Colours present in rgb labels: 1003432 3310765 6148388 7578015 8128526 11612070 14234127 14566981 14895524 15407937 16605960


In [41]:
print(
    "Events that contain more than one connected component:",
    *separated_events[movie_id]
)

Events that contain more than one connected component: [1003432, 3] [6148388, 3] [16605960, 2]


#### !!! the changes here are made interactively on the Napari viewer !!!

the array that gets modified is `rgb_labels_V0[movie_id]`

In [72]:
# check which events are stil separated after changes
# compute number of connected components for each event
list_events = list(np.unique(int_rgb_label))
list_events.remove(0)

print("Events that are still separated in multiple CCs:")

for nb_event in list_events:
    events, n_components = cc3d.connected_components(
        (int_rgb_label == nb_event), connectivity=connectivity, return_N=True
    )
    if n_components > 1:
        print(
            f"Event with colour {nb_event} contains {n_components} connected components, first frame: {list(np.unique((np.where(int_rgb_label == nb_event)[0])))[0]}"
        )

Events that are still separated in multiple CCs:


In [49]:
nb_event = 16605960

print(
    f"frames containing event {nb_event}: {list(np.unique((np.where(int_rgb_label == nb_event)[0])))}"
)

# array containing the separated connected component of the selected event
separted_event_conn_comp = cc3d.connected_components(
    (int_rgb_label == nb_event), connectivity=connectivity, return_N=False
)

viewer = napari.Viewer()
viewer.add_image(movie, name="original_movie")
viewer.add_labels(int_rgb_label, name="int rgb labels", opacity=0.5, visible=False)
viewer.add_labels(class_label, name="class labels", opacity=0.5, visible=False)
viewer.add_labels(
    separted_event_conn_comp, name="separated event connected components", opacity=0.5
)

frames containing event 16605960: [880, 882, 883]


<Labels layer 'separated event connected components' at 0x20f15bfd6a0>

In [50]:
# check slices that contain a given CC for the current event number
cc_idx = 2
np.unique(np.where(separted_event_conn_comp == cc_idx)[0])

array([882, 883], dtype=int64)

In [74]:
# save corrected rgb labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V1.tif"), int_rgb_label
)

#### Run this to interactively apply new thresholds on smooth video + correct RGB labels (V0)

This is applied to the current `movie_id` (and visualised with respect to current `nb_event`)

In [54]:
# open smooth movie
smooth_movies_dir = os.path.join(data_dir, "smoothed_movies")
smooth_movie = load_movies_ids(data_folder=smooth_movies_dir, ids=[movie_id])[movie_id]

In [55]:
# select start and end frames of the event (event belongs to these frames)
start = 872
end = 357

In [56]:
# empty previous and successive frames in smooth movie
smooth_movie[:start] = 0
# smooth_movie[end+1:] = 0

In [68]:
# reset previous threshold
int_rgb_label = np.where(int_rgb_label == white_int, 0, int_rgb_label)

# select event threshold
t = 1.105

# compute binary mask
binary_mask = smooth_movie > t

# add new detected ROIs to rgb_labels_V0[movie_id] (already opened in previous section) as white_int
int_rgb_label = np.where(
    np.logical_and(binary_mask, int_rgb_label == 0), white_int, int_rgb_label
)

In [69]:
# visualise everything and apply changes manually
viewer = napari.Viewer()
viewer.add_image(movie, name="original movie")
viewer.add_labels(int_rgb_label, name="int rgb labels")  # , opacity=0.5)
viewer.add_labels(class_label, name="class labels", opacity=0.5, visible=False)
viewer.add_labels(
    separted_event_conn_comp,
    name="separated event connected components",
    opacity=0.5,
    visible=False,
)

<Labels layer 'separated event connected components' at 0x20f7fa952e0>

In [71]:
# set remaining white_int zones back to zero
int_rgb_label = np.where(int_rgb_label == white_int, 0, int_rgb_label)

#### Run this to detrend current movie and interactively apply new thresholds to annotations

In [719]:
# create a copy of RGB labels V0 to avoid losing changes
int_rgb_label_copy = rgb_labels_V0_bis[movie_id]

In [716]:
smooth_movie_detrend = detrend(smooth_movie, axis=0, bp=[1480])

imageio.volwrite(f"TEST_detrend_movie_{movie_id}.tif", smooth_movie_detrend)

In [818]:
# check which events are stil separated after changes
# compute number of connected components for each event
list_events = list(np.unique(int_rgb_label_copy))
list_events.remove(0)

print("Events that are still separated in multiple CCs:")

for nb_event in list_events:
    events, n_components = cc3d.connected_components(
        (int_rgb_label_copy == nb_event), connectivity=connectivity, return_N=True
    )
    if n_components > 1:
        print(
            f"Event with colour {nb_event} contains {n_components} connected components, first frame: {list(np.unique((np.where(int_rgb_label_copy == nb_event)[0])))[0]}"
        )

Events that are still separated in multiple CCs:


In [829]:
# compute connected components of given event
nb_event = 7522652

print(
    f"frames containing event {nb_event}: {list(np.unique((np.where(int_rgb_label_copy == nb_event)[0])))}"
)

# array containing the separated connected component of the selected event
separted_event_conn_comp = cc3d.connected_components(
    (int_rgb_label_copy == nb_event), connectivity=connectivity, return_N=False
)

# visualise everything and apply changes manually

viewer = napari.Viewer()
viewer.add_image(smooth_movie_detrend, name="original movie")
viewer.add_labels(int_rgb_label_copy, name="int rgb labels", opacity=0.5, visible=False)
viewer.add_labels(class_label, name="class labels", opacity=0.5, visible=False)
viewer.add_labels(
    separted_event_conn_comp,
    name="separated event connected components",
    opacity=0.5,
    visible=True,
)

frames containing event 7522652: [847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902]


<Labels layer 'separated event connected components' at 0x1da5bc19c40>

In [808]:
# select start and end frames of the event (event belongs to these frames)
start = 1770
end = 1492

In [809]:
# empty previous and successive frames in smooth movie
# smooth_movie_detrend_t = np.copy(smooth_movie_detrend)
# smooth_movie_detrend_t[:start] = 0
# smooth_movie_detrend_t[end+1:] = 0
smooth_movie[:start] = 0
# smooth_movie[end+1:] = 0

In [814]:
# reset previous threshold
int_rgb_label_copy = np.where(int_rgb_label_copy == white_int, 0, int_rgb_label_copy)

# select event threshold
t = 2.35

# compute binary mask
# binary_mask = smooth_movie_detrend_t > t
binary_mask = smooth_movie > t

# add new detected ROIs to rgb_labels_V0[movie_id] (already opened in previous section) as white_int
int_rgb_label_copy = np.where(
    np.logical_and(binary_mask, int_rgb_label_copy == 0), white_int, int_rgb_label_copy
)

In [817]:
# set remaining white_int zones back to zero
int_rgb_label_copy = np.where(int_rgb_label_copy == white_int, 0, int_rgb_label_copy)

In [830]:
# save changes to int_rgb_label
int_rgb_label = int_rgb_label_copy

#### Run this to change current CC colours

In [417]:
# change CC colours manually
int_rgb_label = np.where(separted_event_conn_comp == 2, 7, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 3, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 4, 6, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 5, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 6, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 7, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 8, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 9, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 10, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 11, 0, int_rgb_label)
int_rgb_label = np.where(separted_event_conn_comp == 12, 0, int_rgb_label)

In [434]:
# loop over a range of CCs indices
for i in range(2, np.max(separted_event_conn_comp) + 1):
    int_rgb_label = np.where(separted_event_conn_comp == i, 0, int_rgb_label)

## Load corrected "int"-RGB labels and label events

The goal here is to assign a class with value in `{1,2,3,4}` to each event present in the files `XX_corrected_rgb_mask_V1.tif`.

The idea is the following:
- Load "int"-RGB mask V1 : `data/raw_data_and_processing_manual_corr_separated_events_masks/XX_corrected_rgb_mask_V1.tif`
- Where "int"-RGB mask != 0 and label mask != 0, set RGB mask value to label mask value
- Check remaining ROIs with values >= 5 and correct manually
- Check events whose class has to be manually changes (check `separated_events_analysis.xlsx` and discuss with Miguel)

New masks are saved (as "int"-RGB masks) in the directory `data/raw_data_and_processing_manual_corr_separated_events_masks` with name `XX_corrected_label_mask_V0.tif`.

In [75]:
rgb_labels_V1 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V1"
)

In [76]:
white_int = 255 * 255 * 255 + 255 * 255 + 255
print("white colour:", white_int)

white colour: 16646655


### Automatic step: set all ROIs that are already labelled to their class

In [77]:
# create dict to store labelled RGB masks
class_rgb_labels = {}

for movie_id in movie_ids:
    print("Movie", movie_id)
    int_rgb_label = rgb_labels_V1[movie_id]
    class_label = class_labels[movie_id]

    # assert that int_rgb_label does not contain values between 1 and 4
    assert not any(elem in int_rgb_label for elem in [1, 2, 3, 4])

    # set ROIs of int_rgb_label that are labelled to the corresponding value in class_label
    class_rgb_labels[movie_id] = np.where(
        np.logical_and(int_rgb_label != 0, class_label != 0), class_label, int_rgb_label
    )

    """# compute number of connected components for each event
    list_events = list(np.unique(int_rgb_label))
    list_events.remove(0)
    
    # create dict entry to store events that are separated in more than one conn comp
    separated_events[movie_id] = []
    
    for nb_event in list_events:
        events, n_components = cc3d.connected_components((int_rgb_label == nb_event),
                                                         connectivity=connectivity,
                                                         return_N=True
                                                        )
        if n_components > 1:
            print(f"Event with colour {nb_event} contains {n_components} connected components")
            separated_events[movie_id].append([nb_event, n_components])"""

Movie 01
Movie 02
Movie 03
Movie 04
Movie 05
Movie 06
Movie 07
Movie 08
Movie 09
Movie 10
Movie 11
Movie 12
Movie 13
Movie 14
Movie 15
Movie 16
Movie 17
Movie 18
Movie 19
Movie 20
Movie 21
Movie 22
Movie 23
Movie 24
Movie 25
Movie 27
Movie 28
Movie 29
Movie 30
Movie 32
Movie 33
Movie 34
Movie 35
Movie 36
Movie 38
Movie 39
Movie 40
Movie 41
Movie 42
Movie 43
Movie 44
Movie 45
Movie 46


### Correct classes manually in each movie

In [87]:
movie_id = "32"
movie = movies[movie_id]
int_rgb_label = rgb_labels_V1[movie_id]
class_rgb_label = class_rgb_labels[movie_id]
class_label = class_labels[movie_id]

#### !!! the changes here are made interactively on the Napari viewer !!!

the array that gets modified is `class_rgb_labels[movie_id]`

In [93]:
class_rgb_label_colours = list(np.unique(class_rgb_label))
class_rgb_label_colours.remove(0)

print("Colours present in rgb labels:", *class_rgb_label_colours)
print()

# get events that still need to be labelled and slices that contain them
list_events = list(np.unique(class_rgb_label))
for i in range(5):
    if i in list_events:
        list_events.remove(i)

for nb_event in list_events:
    print(
        f"Event with colour {nb_event} is still present in frames: {list(np.unique((np.where(class_rgb_label == nb_event)[0])))}"
    )

Colours present in rgb labels: 1 3 4



In [92]:
# nb_event = 7757222

# mask of the selected event
# event_mask = class_rgb_label == nb_event
# print(f"frames containing event {nb_event}: {list(np.unique((np.where(event_mask)[0])))}")

viewer = napari.Viewer()
viewer.add_image(movie, name="original_movie")
viewer.add_labels(int_rgb_label, name="int rgb labels", opacity=0.5, visible=False)
viewer.add_labels(class_label, name="class labels", opacity=0.5, visible=False)
# viewer.add_labels(event_mask, name='event mask', opacity=0.5, visible=False)
viewer.add_labels(class_rgb_label, name="class rgb labels", opacity=0.5, visible=True)

<Labels layer 'class rgb labels' at 0x20ff03d5340>

In [94]:
# save corrected rgb labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V0.tif"), class_rgb_label
)

#### Run this to automatically change some `nb_event` colours

In [90]:
# change CC colours manually
nb_event = 11612070
new_colour = 3
class_rgb_label = np.where(class_rgb_label == nb_event, new_colour, class_rgb_label)

## Check: each # event contains only 1 class label

Check that each event correspond only to one type of event, otherwise split # event in two (manually).

Changes are tracked in file `data/raw_data_and_processing_manual_corr_separated_events_masks/separated_events_analysis.xlsx`.

In order to do this, for each movie:
- load `events_mask = XX_corrected_rgb_mask_V1.tif`
- load `classes_mask = XX_corrected_label_mask_V0.tif`
- for each # event in `events_mask` check number of classes
- if number of classes > 1, possible solutions:
    - split event
    - merge class labelling
    - remove one of the class ROIs
    - merge class ROIs to another event
    - ...
    
New event labels are saved as `XX_corrected_rgb_mask_V2.tif` and new class labels are saved as `XX_corrected_label_mask_V1.tif`


In [8]:
rgb_labels_V1 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V1"
)
class_labels_V0 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V0"
)

In [10]:
for movie_id in movie_ids:
    print("Movie", movie_id)
    events_mask = rgb_labels_V1[movie_id]
    classes_mask = class_labels_V0[movie_id]

    list_events = list(np.unique(events_mask))
    list_events.remove(0)

    for event_id in list_events:
        count_classes_in_roi(events_mask, classes_mask, event_id)

Movie 01
Movie 02
Movie 03
Movie 04
Movie 05
Event 7 contains 2 classes: [1, 3]
Event 2339872 contains 2 classes: [1, 3]
Movie 06
Movie 07
Movie 08
Movie 09
Movie 10
Movie 11
Event 12995516 contains 2 classes: [3, 4]
Movie 12
Movie 13
Movie 14
Movie 15
Movie 16
Movie 17
Movie 18
Event 6952654 contains 2 classes: [1, 3]
Event 12809738 contains 2 classes: [1, 3]
Movie 19
Event 7 contains 2 classes: [3, 4]
Movie 20
Event 1199049 contains 2 classes: [1, 3]
Event 1300734 contains 2 classes: [1, 3]
Event 5180168 contains 3 classes: [1, 3, 4]
Event 16326961 contains 2 classes: [1, 3]
Movie 21
Event 12646857 contains 2 classes: [3, 4]
Movie 22
Event 6116268 contains 2 classes: [2, 3]
Movie 23
Event 8134414 contains 2 classes: [1, 3]
Movie 24
Event 336152 contains 2 classes: [1, 3]
Movie 25
Movie 27
Movie 28
Movie 29
Event 9022347 contains 2 classes: [1, 3]
Movie 30
Event 11 contains 2 classes: [1, 3]
Event 2673702 contains 2 classes: [2, 3]
Event 10417792 contains 2 classes: [1, 3]
Event 11718

### Correct events manually in each movie

In [209]:
movie_id = "46"
movie = movies[movie_id]
events_mask = rgb_labels_V1[movie_id]
classes_mask = class_labels_V0[movie_id]

In [210]:
events_mask_colours = list(np.unique(events_mask))
events_mask_colours.remove(0)

print("Colours present in rgb labels:", *events_mask_colours)

Colours present in rgb labels: 5 6 7 871378 933260 978479 2225770 2952944 3117976 3893075 4888731 5669422 5777632 6040103 6240425 6622415 6773590 7617704 7695995 7726524 8469808 9615660 9831321 10323694 10340821 10857031 11307398 11309117 11577598 12656347 13871294 14430244 14446044 14800953 14816150 15021344 15468584 15550557 15965776


#### !!! the changes here are made interactively on the Napari viewer !!!

the array that gets modified is `events_mask = rgb_labels_V1[movie_id]` **OR** `classes_mask = class_labels_V0[movie_id]`

In [217]:
list_events = list(np.unique(events_mask))
list_events.remove(0)

for event_id in list_events:
    event_class_mask = np.where(events_mask == event_id, classes_mask, 0)

    # count number of classes
    event_classes = list(np.unique(event_class_mask))
    event_classes.remove(0)

    n_classes = len(event_classes)

    if n_classes > 1:
        print(f"Event {event_id} contains {n_classes} classes: {event_classes}")

In [215]:
event_id = 3117976

# mask of the selected event
event_class_mask = np.where(events_mask == event_id, classes_mask, 0)
print(
    f"frames containing event {event_id}: {list(np.unique((np.where(event_class_mask)[0])))}"
)

viewer = napari.Viewer()
viewer.add_image(movie, name="original_movie")
viewer.add_labels(events_mask, name="events labels", opacity=0.5, visible=False)
viewer.add_labels(classes_mask, name="new classes labels", opacity=0.5, visible=False)
viewer.add_labels(event_class_mask, name="single event mask", opacity=0.5, visible=True)

frames containing event 3117976: [726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 9

<Labels layer 'single event mask' at 0x29e02f2a400>

In [216]:
event_type = 4
list(np.unique(np.where(event_class_mask == event_type)[0]))

[]

In [218]:
# save corrected rgb labels and class labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V2.tif"), events_mask
)
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V1.tif"), classes_mask
)

#### Run this to automatically remove part of events (that have wrong label)
I.e. if an event has two class labels and one of them should be removed from `classes_mask`, it is important to also remove the corresponging # event labels on `events_mask`.

In [214]:
# remove class and event ROIs
class_to_remove = 4
mask_to_remove = event_class_mask == class_to_remove
classes_mask = np.where(mask_to_remove, 0, classes_mask)
events_mask = np.where(mask_to_remove, 0, events_mask)

#### Run this if the ROIs already have the right class, but the # event is wrong

In [181]:
class_to_transfer = 3
correct_event_id = 4611557

events_mask = np.where(
    event_class_mask == class_to_transfer, correct_event_id, events_mask
)

## Check if RGB and class masks match

In [8]:
rgb_labels_V2 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V2"
)
class_labels_V1 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V1"
)

In [19]:
for movie_id in movie_ids:
    events_mask = rgb_labels_V2[movie_id]
    classes_mask = class_labels_V1[movie_id]

    # check that events_mask and classes_mask match
    if not (events_mask.astype(bool) == classes_mask.astype(bool)).all():
        print(f"RGB and class masks do not match in movie {movie_id}")

RGB and class masks do not match in movie 07


In [22]:
movie_id = "07"

events_mask = rgb_labels_V2[movie_id]
classes_mask = class_labels_V1[movie_id]

wrong_mask = np.where((events_mask.astype(bool) == classes_mask.astype(bool)), 0, 1)
print(np.unique(np.where(wrong_mask)[0]))

viewer = napari.Viewer()
viewer.add_image(movies[movie_id], name="original_movie")
viewer.add_labels(events_mask, name="events labels", opacity=0.5, visible=False)
viewer.add_labels(classes_mask, name="classes labels", opacity=0.5, visible=False)
viewer.add_labels(wrong_mask, name="wrong labels", opacity=0.5, visible=True)

[]


<Labels layer 'wrong labels' at 0x1eb003a63d0>

In [23]:
# save corrected rgb labels and class labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V2.tif"), events_mask
)
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V1.tif"), classes_mask
)

## Check whether events touch each other
Check if each CC in the rbg mask contains at most one # event.

In order to do this, for each movie:
- load `events_mask = XX_corrected_rgb_mask_V2.tif`
- load `classes_mask = XX_corrected_label_mask_V1.tif`
- compute CCs in `events_mask` and compute number of events in each of them
- if # events > 1 in a CC, possible solutions:
    - check if it is okay
    - or separate events manually
    
New event labels are saved as `XX_corrected_rgb_mask_V3.tif` and new class labels are saved as `XX_corrected_label_mask_V2.tif`

Changes are tracked in file `data/raw_data_and_processing_manual_corr_separated_events_masks/separated_events_analysis.xlsx`.

In [24]:
rgb_labels_V2 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V2"
)
class_labels_V1 = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V1"
)

In [38]:
connectivity = 26

ccs_list = {}
for movie_id in movie_ids:
    ccs_list[movie_id] = []
    print("Movie", movie_id)
    events_mask = rgb_labels_V2[movie_id]
    classes_mask = class_labels_V1[movie_id]

    # check that events_mask and classes_mask match
    assert (events_mask.astype(bool) == classes_mask.astype(bool)).all()

    # extract connected components in event masks
    ccs, n_ccs = cc3d.connected_components(
        events_mask.astype(bool), connectivity=connectivity, return_N=True
    )

    for cc_id, cc in cc3d.each(ccs, binary=True, in_place=True):
        # for each CC, count number of events in it
        events = np.where(cc, events_mask, 0)
        list_events = list(np.unique(events))
        list_events.remove(0)

        if len(list_events) > 1:
            ccs_list[movie_id].append(list_events)
            print(f"Events {list_events} belong to the same connected component.")

Movie 01
Events [5047384, 11014150] belong to the same connected component.
Events [2177621, 4641927, 13703862, 14663886] belong to the same connected component.
Movie 02
Events [6, 4192011, 7529393, 10252693] belong to the same connected component.
Movie 03
Events [102422, 5831824, 6204449] belong to the same connected component.
Movie 04
Movie 05
Events [10, 2339872] belong to the same connected component.
Events [882547, 896217] belong to the same connected component.
Events [463234, 10537788] belong to the same connected component.
Events [7, 11357165] belong to the same connected component.
Events [5268606, 6415686] belong to the same connected component.
Movie 06
Events [8071199, 8843435] belong to the same connected component.
Events [2199665, 14807770] belong to the same connected component.
Movie 07
Events [2945582, 16370811] belong to the same connected component.
Events [3573375, 5525351, 12982121] belong to the same connected component.
Movie 08
Movie 09
Movie 10
Events [28

### Correct events manually in each movie

In [346]:
movie_id = "14"
movie = movies[movie_id]
events_mask = rgb_labels_V2[movie_id]
classes_mask = class_labels_V1[movie_id]

#### !!! the changes here are made interactively on the Napari viewer !!!

the array that gets modified is `events_mask = rgb_labels_V2[movie_id]` **AND** `classes_mask = class_labels_V1[movie_id]`

In [7]:
# compute events that are in same CC
ccs_to_check = ccs_list[movie_id]
for cc in ccs_to_check:
    print(cc)

NameError: name 'ccs_list' is not defined

In [341]:
cc = [3117976, 5777632, 7726524, 11309117]

to_check_mask = np.zeros_like(events_mask)
for nb_event in cc:
    to_check_mask += np.where(events_mask == nb_event, events_mask, 0)

print(f"frames containing events {cc}: {list(np.unique((np.where(to_check_mask)[0])))}")

viewer = napari.Viewer()
viewer.add_image(movie, name="original_movie")
viewer.add_labels(events_mask, name="events labels", opacity=0.5, visible=False)
viewer.add_labels(classes_mask, name="new classes labels", opacity=0.5, visible=False)
viewer.add_labels(to_check_mask, name="touching events", opacity=0.5, visible=True)

frames containing events [3117976, 5777632, 7726524, 11309117]: [726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 

<Labels layer 'touching events' at 0x1efe874eaf0>

In [330]:
event_id = 14816150
np.unique(np.where(events_mask == event_id)[0])

array([110, 111, 112, 113], dtype=int64)

In [348]:
# save corrected rgb labels and class labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V3.tif"), events_mask
)
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V2.tif"), classes_mask
)

#### Run this to transfer some ROIs from a # event to another # event

In [337]:
old_n_event = 11309117
new_n_event = 3117976

events_mask = np.where(events_mask == old_n_event, new_n_event, events_mask)

#### Run this to change class of a # event

In [340]:
n_event = 3117976
new_class = 2

classes_mask = np.where(events_mask == n_event, new_class, classes_mask)

## Change class of events denoted in file excel and discussed with Miguel

First time I load **XX_corrected_rgb_mask_V3.tif** and **XX_corrected_label_mask_V2.tif** and save them as **XX_corrected_rgb_mask_V4.tif** and **XX_corrected_label_mask_V3.tif**. These will be the final versions of the files.

TODO:
- check remarks done in `data/raw_data_and_processing_manual_corr_separated_events_masks/separated_events_analysis.xlsx`
- discuss with Miguel
- correct events
- train UNet

In [10]:
# only for first run
# rgb_labels = load_annotations_ids(data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V3")
# class_labels = load_annotations_ids(data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V2")

rgb_labels = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V4"
)
class_labels = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V3"
)

In [11]:
for movie_id in movie_ids:
    events_mask = rgb_labels[movie_id]
    classes_mask = class_labels[movie_id]

    # check that events_mask and classes_mask match
    if not (events_mask.astype(bool) == classes_mask.astype(bool)).all():
        print(f"RGB and class masks do not match in movie {movie_id}")
        print(np.where(events_mask.astype(bool) != classes_mask.astype(bool)))

### Correct events manually in each movie

In [105]:
movie_id = "36"
movie = movies[movie_id]
smooth_movie = smooth_movies[movie_id]
events_mask = rgb_labels[movie_id]
classes_mask = class_labels[movie_id]

#### Compute stronger smoothing on movie

In [111]:
very_smooth_movie = moving_average(smooth_movie, k=5)

In [115]:
cc = [12753098]

to_check_mask = np.zeros_like(events_mask)
for nb_event in cc:
    to_check_mask += np.where(events_mask == nb_event, events_mask, 0)

print(f"frames containing events {cc}: {list(np.unique((np.where(to_check_mask)[0])))}")

viewer = napari.Viewer()
viewer.add_image(  # smooth_movie,
    very_smooth_movie, name="smooth movie", colormap=("colors", cmap)
)

# viewer.add_image(movie,
#                 name='original_movie')

viewer.add_labels(events_mask, name="events labels", opacity=0.5, visible=False)

viewer.add_labels(classes_mask, name="classes labels", opacity=0.5, visible=False)

viewer.add_labels(to_check_mask, name="events to check", opacity=0.5, visible=True)

frames containing events [12753098]: [708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724]


<Labels layer 'events to check' at 0x2415ce9a0d0>

In [16]:
# search fon an event
event_id = 8106706
np.unique(np.where(events_mask == event_id)[0])

array([], dtype=int64)

In [98]:
# get list of events
list_events = list(np.unique(events_mask))
list_events.remove(0)
print(list_events)

[5, 6, 223058, 295664, 329779, 448200, 688987, 1246149, 2109825, 2352444, 2495642, 2889855, 2924334, 2964884, 3657769, 4611557, 4971212, 5066441, 5439214, 5728718, 6360106, 6398526, 6502712, 6625189, 7028853, 7244004, 7248105, 8245031, 8249378, 8508743, 8794858, 9023255, 9323422, 9967906, 9969732, 10065488, 10287463, 10899810, 11051320, 11067066, 11090715, 11106725, 11160070, 11584814, 12114165, 13089761, 13356009, 13395119, 13466553, 13578846, 13712710, 14254612, 14534109, 14555138, 14651920, 14973194, 15267821, 15345751, 15384501, 15574667, 16090447, 16416643]


In [104]:
# save corrected rgb labels and class labels on disk
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V4.tif"), events_mask
)
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V3.tif"), classes_mask
)

#### Run this to change class of a # event

In [113]:
n_event = 0
new_class = 0

classes_mask = np.where(events_mask == n_event, new_class, classes_mask)

#### Load previous version of a mask of current movie

In [89]:
v = "V2"  # mask version
old_rgb_mask = load_annotations_ids(
    data_folder=corr_dir, ids=[movie_id], mask_names="corrected_rgb_mask_" + v
)[movie_id]

In [93]:
# search for an event
find_event_id = 6344796
np.unique(np.where(old_rgb_mask == find_event_id)[0])

array([1388, 1389, 1390, 1391, 1392, 1393], dtype=int64)

In [94]:
# recreate previously deleted event
del_event_id = 6344796
events_mask = np.where(old_rgb_mask == del_event_id, del_event_id, events_mask)

#### Re-save all movies for safety

In [101]:
for movie_id in movie_ids:
    events_mask = rgb_labels[movie_id]
    classes_mask = class_labels[movie_id]

    # save corrected rgb labels and class labels on disk
    imageio.volwrite(
        os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V4.tif"), events_mask
    )
    imageio.volwrite(
        os.path.join(corr_dir, movie_id + "_corrected_label_mask_V3.tif"), classes_mask
    )

## Add new events created by Miguel in movie 07

Load **XX_corrected_rgb_mask_V4.tif** and **XX_corrected_label_mask_V3.tif** and save as **XX_corrected_rgb_mask_V4.tif** and **XX_corrected_label_mask_V3.tif** (stil part of the last check of events in general).

Take note of changes in `data/raw_data_and_processing/manual_corr_separated_events_masks/separated_events_analysis.xlsx` (tab "Final check").


In [28]:
white_int = 255 * 255 * 255 + 255 * 255 + 255
print("white colour:", white_int)

white colour: 16646655


In [18]:
movie_id = "07"

In [16]:
rgb_labels = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_rgb_mask_V4"
)
new_class_labels = load_annotations_ids(
    data_folder=corr_dir, ids=movie_ids, mask_names="corrected_label_mask_V3"
)

In [19]:
smooth_movie = smooth_movies[movie_id]
events_mask = rgb_labels[movie_id]
classes_mask = new_class_labels[movie_id]

original_classes_mask = class_labels[movie_id]

# check that events_mask and classes_mask match
if not (events_mask.astype(bool) == classes_mask.astype(bool)).all():
    print(f"RGB and class masks do not match in movie {movie_id}")

### Compute difference in class mask between V4 and new mask created by Miguel

In [25]:
diff = np.where(original_classes_mask, 1, 0) - np.where(classes_mask, 1, 0)

print("Frames where V4 and Miguel's masks differ:", np.unique(np.where(diff)[0]))

Frames where V4 and Miguel's masks differ: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237

In [27]:
viewer = napari.Viewer()
viewer.add_image(smooth_movie, name="smooth movie", colormap=("colors", cmap))

viewer.add_labels(events_mask, name="events mask", opacity=0.3, visible=False)

viewer.add_labels(classes_mask, name="new classes mask", opacity=0.5, visible=False)

viewer.add_labels(
    original_classes_mask, name="Miguels classes mask", opacity=0.5, visible=False
)

viewer.add_labels(diff, name="differences", opacity=0.5, visible=True)

<Labels layer 'differences' at 0x1f658284fa0>

### Analyse ROIs added by Miguel

In [34]:
# set added ROIs to `white_int`
events_mask[diff == 1] = white_int

In [65]:
print("Frames with new ROIs:", np.unique(np.where(events_mask == white_int)[0]))

Frames with new ROIs: []


In [70]:
viewer = napari.Viewer()
viewer.add_image(smooth_movie, name="smooth movie", colormap=("colors", cmap))

viewer.add_labels(events_mask, name="events mask", opacity=0.5, visible=True)

viewer.add_labels(classes_mask, name="new classes mask", opacity=0.5, visible=False)

viewer.add_labels(
    original_classes_mask, name="Miguels classes mask", opacity=0.5, visible=False
)

viewer.add_labels(diff, name="differences", opacity=0.5, visible=False)

<Labels layer 'differences' at 0x1f60fc27490>

#### Run this to change number of a # event

In [53]:
n_event = white_int
new_n_event = 12982121

events_mask = np.where(events_mask == n_event, new_n_event, events_mask)

#### Run this to compute number of connected component of events

In [56]:
# connectivity of the connected components
connectivity = 26

In [60]:
assert not white_int in events_mask

# compute number of connected components for each event
list_events = list(np.unique(events_mask))
list_events.remove(0)

for nb_event in list_events:
    events, n_components = cc3d.connected_components(
        (events_mask == nb_event), connectivity=connectivity, return_N=True
    )
    if n_components > 1:
        print(
            f"Event with colour {nb_event} contains {n_components} connected components"
        )

### Analyse ROIs "removed" by Miguel

In [61]:
print("Frames containing ROIs removed by Miguel", np.unique(np.where(diff == -1)[0]))

Frames containing ROIs removed by Miguel [110 112 297 300 303 305 307 308 311 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 478]


In [63]:
viewer = napari.Viewer()
viewer.add_image(smooth_movie, name="smooth movie", colormap=("colors", cmap))

viewer.add_labels(events_mask, name="events mask", opacity=0.5, visible=False)

viewer.add_labels(classes_mask, name="new classes mask", opacity=0.5, visible=False)

viewer.add_labels(
    original_classes_mask, name="Miguels classes mask", opacity=0.5, visible=False
)

viewer.add_labels(diff == -1, name="removed ROIs", opacity=0.5, visible=True)

<Labels layer 'removed ROIs' at 0x1f60e34f3a0>

### Set classes to new event ROIs

In [71]:
n_event = 0
new_class = 0

classes_mask = np.where(events_mask == n_event, new_class, classes_mask)

In [72]:
print(
    "Frames that still need to be classified:",
    np.unique(np.where(classes_mask == white_int)[0]),
)

Frames that still need to be classified: []


### Save corrected version of movie 07 on disk

In [74]:
# check that class and event masks match
(events_mask.astype(bool) == classes_mask.astype(bool)).all()

True

In [73]:
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_rgb_mask_V4.tif"), events_mask
)
imageio.volwrite(
    os.path.join(corr_dir, movie_id + "_corrected_label_mask_V3.tif"), classes_mask
)

28/04/2023
## Correct manually movie 34

Ho notato che le annotazioni di certe waves nel movie 34 non hanno senso (ad esempio alcune sono separate e non formano una wave di per sé --> label con 4)

Files modificati:
- `data/raw_data_and_processing/manual_corr_separated_events_masks/final_masks/34_class_label.tif`
- `data/raw_data_and_processing/manual_corr_separated_events_masks/final_masks/34_event_label.tif`

Changes are tracked in file `data/raw_data_and_processing_manual_corr_separated_events_masks/separated_events_analysis.xlsx`.

In [48]:
movie_id = "34"

In [54]:
final_masks_dir = os.path.join(corr_dir, "final_masks")
event_label = load_annotations_ids(
    data_folder=final_masks_dir, ids=[movie_id], mask_names="event_label"
)[movie_id]
class_label = load_annotations_ids(
    data_folder=final_masks_dir, ids=[movie_id], mask_names="class_label"
)[movie_id]

In [58]:
print("Number of annotated waves (should be == 4):")

waves_mask = np.where(class_label == 2, event_label, 0)
print(np.unique(waves_mask), "-->", len(np.unique(waves_mask)) - 1)

Number of annotated waves (should be == 4):
[ 0 17 20 24 26 31 32] --> 6


In [68]:
viewer = napari.Viewer()
viewer.add_image(smooth_movies["34"], name="smooth movie", colormap=("colors", cmap))

viewer.add_labels(waves_mask, name="waves mask", opacity=0.5, visible=True)

viewer.add_labels(class_label, name="classes mask", opacity=0.5, visible=True)

<Labels layer 'classes mask' at 0x25b0b4a4e20>

In [67]:
# set waves labelled with 17 and 31 to undefined ROIs
new_class_label = np.where(waves_mask == 17, 4, class_label)
new_class_label = np.where(waves_mask == 31, 2, new_class_label)

In [69]:
viewer.add_labels(new_class_label, name="new classes mask", opacity=0.5, visible=True)

<Labels layer 'new classes mask' at 0x25b10479310>

In [70]:
final_masks_dir

'..\\data\\raw_data_and_processing\\manual_corr_separated_event_masks\\final_masks'

In [71]:
imageio.volwrite(
    os.path.join(final_masks_dir, movie_id + "_class_label.tif"), new_class_label
)